In [19]:
import pandas as pd
import numpy as np

In [43]:
ds_train = pd.read_csv('ds/train.csv')
ds_songs = pd.read_csv('ds/songs.csv')
ds_songs_extra_info = pd.read_csv('ds/song_extra_info.csv')
ds_members = pd.read_csv('ds/members.csv')

In [44]:
# ds_train = ds_train.merge(ds_members,on='msno',how='left')
# ds_train = ds_train.merge(ds_songs,on='song_id',how='left')
# ds_train = ds_train.fillna(0)

### Беру только 10% от датасета, потому что построение предложений идут очень долго =(

In [45]:
ds_train = ds_train.sample(frac=0.1)

In [46]:
X_train = ds_train.sample(frac=0.8)
X_test = ds_train.drop(X_train.index)

### В качестве предложений(sentence) для каждого юзера буду рассматривать список песен, которые он лайкнул. Где айдишки песен - слова для word2vec

In [47]:
X_train_pos = X_train[X_train['target'] == 1]

In [48]:
from tqdm.notebook import tqdm, trange

In [49]:
user_pos_history = [X_train_pos[X_train_pos['msno'] == user]['song_id'].astype(str).tolist() for user in tqdm(users)]

In [50]:
from gensim.models import Word2Vec

In [54]:
lengths = [len(arr) for arr in user_pos_history]

8.639275644565881
3.0


In [70]:
w2v = Word2Vec(window=8, sg=1, hs=0, negative=3, alpha=0.1, min_alpha=0.0001)

In [71]:
w2v.build_vocab(user_pos_history)

In [72]:
w2v.train(user_pos_history, total_examples=len(user_pos_history), epochs=30)

(6794741, 8916510)

In [74]:
ds_songs = ds_songs.merge(ds_songs_extra_info,on='song_id',how='left')

In [124]:
ds_songs.shape

(2296320, 9)

In [78]:
print(w2v)

Word2Vec(vocab=9038, vector_size=100, alpha=0.1)


In [119]:
def get_similars(name):
    sims = w2v.wv.most_similar_cosmul(name)
    data = []
    for song, similarity in sims:
        d = ds_songs[ds_songs['song_id'] == song].to_dict('records')[0]
        d['similarity'] = similarity
        data.append(d)
    return pd.DataFrame(data)

In [163]:
# name = '0ee/D6HjX0jSytUn3v58C0x6zEzN4bcPq3WlqvrG/Xo=' # heathens, Twenty one pilots
name = 'e/upMhPFcXl3Rxq0Us1abr0FcThlZOeqKJhBobmltBU=' # Shape of you
print(name)
get_similars(name)

e/upMhPFcXl3Rxq0Us1abr0FcThlZOeqKJhBobmltBU=


,song_id,song_length,genre_ids,artist_name,composer,lyricist,language,name,isrc,similarity
0,JiHWUXkERiCPai7GKXVal+Cns8SWC8Ft0+5NKSQdbfU=,234370,465,Austin Mahone,NaN,NaN,52.0,Shake It For Me (feat. 2 Chainz),QMRSZ1601336,0.749492
1,p0vznXPDpqFXtJzcvx4f9NxqLDC+1PQ9/Prr8kUdSuc=,161149,1609,The Chainsmokers,Andrew Taggart| Shaun Frank| Frederic Kennett|...,Andrew Taggart| Shaun Frank| Frederic Kennett|...,52.0,Closer,USQX91602095,0.736199
2,ulBJRABKso5kGxygDdVCfTpdXaSIIzyLqPsU+ciOKm8=,331395,1259,J. Cole,Jermaine Cole,NaN,52.0,Change,QMJMT1601086,0.736063
3,SVPVLZg1t7+aQoqFfQPs5lEl4iHf/DYf8hB/bbVyr88=,338860,2022|359,Slowdive,NaN,NaN,52.0,Star Roving,USJ5G1713202,0.731613
4,DyglzNccElIQTMtiKRbnHiYdKO+tEZCNA4G5DTZs4FY=,237959,465,Wham!,George Michael,George Michael,52.0,A Different Corner,GBBBM8600001,0.722280
5,1GI0GC3w9c0tDD2s0qP15DxrLUs9JX9XW6fKbY0rBZ8=,220496,1609,DJ Snake,William Grigahcine| Adio Marchant| A. Kleinstub,NaN,52.0,Middle,USUM71515784,0.714463
6,WMLsyMjcYUps04F3ZnRjvz7LqNmjGY76hl/D8V4ExgQ=,192261,1609,Grey,Kyle Trewartha| Michael Trewartha| Sasha Yatch...,NaN,52.0,I Miss You,USUM71700035,0.712479
7,glS5BF8YGt1ogLMHKDjfW9rlF+1YxkBhKt/6N70STR0=,193840,139,José James,José James| Antario Dion Holmes,NaN,52.0,Always There,USUM71614569,0.711586
8,v/3onppBGoSpGsWb8iaCIO8eX5+iacbH5a4ZUhT7N54=,161175,1616|1609,Alan Walker,Alan Walker| Jesper Borgen| Anders Froen| Gunn...,Alan Walker| Jesper Borgen| Anders Froen| Gunn...,52.0,Alone,NOG841617010,0.708199
9,NGem2a+lx/wBSRrCNLD5MegHBlls69v78Mv9cjHHy9s=,328237,465,George Michael,NaN,NaN,52.0,You Have Been Loved,GBARL0601621,0.706002


In [176]:
song_data_name_author = []
songs_in_dict = list(w2v.wv.key_to_index.keys())
# for song_id in tqdm(w2v.wv.key_to_index.keys()):
#     print(song_id, ds_songs[ds_songs['song_id'] == song_id]['name'], ds_songs[ds_songs['song_id'] == song_id]['artist_name'])
#     song_data_name_author.append((song_id, ds_songs[ds_songs['song_id'] == song_id]['name'], ds_songs[ds_songs['song_id'] == song_id]['artist_name']))

In [173]:
# w2v.wv.get_vector("e/upMhPFcXl3Rxq0Us1abr0FcThlZOeqKJhBobmltBU=",norm=True)

In [175]:
y_train = X_train['target']
y_test = X_test['target']
X_train = X_train.drop(columns=['target'])
X_test = X_test.drop(columns=['target'])

In [184]:
def get_user_embeddings(X, y):
    embeddings = dict()
    count = dict()
    for (index, row), target in tqdm(zip(X_train.iterrows(), y_train)):
        if target == 1:
            user_id = row['msno']
            song_id = row['song_id']
            if song_id in songs_in_dict:
                if user_id not in count:
                    count[user_id] = 0
                if user_id not in embeddings:
                    embeddings[user_id] = np.zeros(100)
                embeddings[user_id] += w2v.wv.get_vector(song_id, norm=True)
                count[user_id] += 1
    for user_id in embeddings.keys():
        embeddings[user_id] /= count[user_id]
    return embeddings    

In [185]:
user_embeddings = get_user_embeddings(X_train, y_train)

In [201]:
def get_user_song_dot(user_id, song_id):
    if user_id not in user_embeddings:
        return 0
    if song_id not in songs_in_dict:
        return 0
    uv = user_embeddings[user_id]
    sv = w2v.wv.get_vector(song_id)
    
    u_n = np.linalg.norm(uv)
    s_n = np.linalg.norm(sv)
    return np.dot(uv / u_n, sv / s_n)

In [202]:
X_test_c = X_test.copy()

In [203]:
target_res = []
for index, row in tqdm(X_test_c.iterrows()):
    user_id = row['msno']
    song_id = row['song_id']
    target_res.append(get_user_song_dot(user_id, song_id))

### Результат хуже, чем у бустинга, вероятно мало данных, и много пропущенных песен

In [204]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, target_res)

0.6180597257256804